# Advanced Neural Network Assignment

     본격적인 모델링에 앞서 사용할 패키지 및 모듈 import 등 전반적인 준비와 데이터를 훑어보는 과정을 진행한다. 필자의 경우, 코랩이 처음이기에 드라이브 마운트와 같은 준비가 필요했다.

     이후 더 사용하게 될 프레임워크 등은 과제를 진행하면서 차근차근 준비하기로 한다. 아래의 과정은 모듈 import 및 데이터 확인 과정이다.

## Basic Models

In [ ]:
!ls '/content/drive/'

'My Drive'


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/train_df.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/test_df.csv')

In [ ]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test.head()

,Unnamed: 0,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,57808,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,35755,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,48968,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train.shape, test.shape

((42000, 785), (18000, 785))

In [ ]:
train['label'].value_counts()

3    4246
4    4232
7    4228
0    4202
1    4196
6    4193
8    4180
5    4177
2    4177
9    4169
Name: label, dtype: int64

     위처럼 train data의 label 값을 확인해본 결과, 0 ~ 9 까지의 10가지 값을 가지고 있음을 확인했다. 주의해야할 점은 kaggle의 overview에서는 1 ~ 10의 값으로 kannada digits을 분류하고 있으므로 혼동하지 않도록 주의하자.

     불러온 train 데이터를 아래와 같이 X,y로 분류해주고, test 데이터는 후에 사용될 것이므로 X_test로 따로 분류해둔다.

     이미지의 경우, 0 ~ 255의 픽셀값을 가지므로 255로 나눠서 정규화를 시켜준다. 이 과정 이후, X_test data 와 X의 min, max 값을 찍어보면 0과 1로 이루어진 것을 확인할 수 있다.

In [ ]:
y = train['label'].values
X = train.drop('label',axis = 1).values
X_test = test.iloc[:,1:].values

X = X/255
X_test = X_test/255

In [ ]:
print(X.max())
print(X_test.max())
print(X.min())
print(X_test.min())

1.0
1.0
0.0
0.0


     train_test_split을 통해 X와 y를 각각 train과 val로 나눠준다. 여기서 train은 모델을 학습시킬 때 사용하고, val은 모델을 평가할 때 사용할 데이터들이다.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size = 0.2)

print(X_train.shape,y_train.shape)
print(X_val.shape, y_val.shape)

(33600, 784) (33600,)
(8400, 784) (8400,)


     본격적인 모델을 만들기 위해 프레임워크를 사용한다. 여기서는 텐서플로우를 사용했다. 

In [ ]:
!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
from tensorflow import keras

     |████████████████████████████████| 380.5MB 50kB/s 
     |████████████████████████████████| 501kB 38.9MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 4.3MB 41.9MB/s 


     우선 가장 기본적인 모델은 참고용으로 줬었던 노트북대로 만들어보기로 했다. 그 모델과 차이점은 layer를 한층 더 쌓았다는 점이다.

     활성함수는 relu를 사용했고, optimizer는 adam을 사용해 신경망을 구축했다. 그리고 batch size는 100으로, epoch는 20으로 모델을 학습시켰다. 이 때 accuracy는 0.988정도가 나왔다. 처음 시도 치고 굉장히 높게 나왔다고 생각했다.

     이 모델을  base 모델로 잡고, 활성함수를 바꾸거나, optimizer를 바꾸거나, Dropout이나 BatchNormalization을 진행해보기로 했다. 진행 과정에서 많은 모델들을 만들어서 그 중 가장 눈에 띄는 것들만 남기기로 했다. (gridsearch를 하지 않은 이유는 조정을 통해 결과를 직접 보기 위함이다.)

     우선 활성함수를 바꿔봤다.

In [21]:
model = keras.Sequential([
                          keras.layers.Dense(512, activation = 'relu'),
                          keras.layers.Dropout(0.3),
                          keras.layers.Dense(256, activation = 'relu'),
                          keras.layers.Dropout(0.3),
                          keras.layers.Dense(128, activation = 'relu'),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(10, activation = 'softmax')
])

model.compile(optimizer='adam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model.fit(X_train, y_train, epochs = 20, batch_size = 100)
model.evaluate(X_val, y_val)

Epoch 1/20
336/336 [==============================] - 1s 4ms/step - loss: 0.2745 - accuracy: 0.9164
Epoch 2/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0931 - accuracy: 0.9717
Epoch 3/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0611 - accuracy: 0.9823
Epoch 4/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0464 - accuracy: 0.9865
Epoch 5/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0366 - accuracy: 0.9887
Epoch 6/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0297 - accuracy: 0.9908
Epoch 7/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0271 - accuracy: 0.9922
Epoch 8/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0240 - accuracy: 0.9924
Epoch 9/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0242 - accuracy: 0.9921
Epoch 10/20
336/336 [==============================] - 1s 4ms/step - loss: 0.0191 - accuracy: 0.9945

[0.05764182284474373, 0.9888095259666443]

In [ ]:
model3 = keras.Sequential([
                          keras.layers.Dense(512),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(256),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(128),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(10,activation = 'softmax')
])

model3.compile(optimizer='adam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model3.fit(X_train, y_train, epochs = 20, batch_size = 100)
model3.evaluate(X_val, y_val)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 2s 55us/sample - loss: 0.2358 - accuracy: 0.9269
Epoch 2/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.0811 - accuracy: 0.9765
Epoch 3/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.0547 - accuracy: 0.9837
Epoch 4/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.0414 - accuracy: 0.9871
Epoch 5/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.0317 - accuracy: 0.9898
Epoch 6/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.0278 - accuracy: 0.9914
Epoch 7/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.0259 - accuracy: 0.9921
Epoch 8/20
33600/33600 [==============================] - 1s 39us/sample - loss: 0.0224 - accuracy: 0.9932
Epoch 9/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.0191 - accuracy: 0.9936
Epoch 10/20
33

[0.06775930667477972, 0.9866667]

     relu, elu, sigmoid 등 여러 함수들을 사용해봤지만 대부분은 비슷비슷한 성능이 나왔다. 그래도 그 중 조금이라도 accuracy가 상승한 모델은 leakyrelu를 사용한 모델이었다. 

     leakyrelu의 경우, 위의 모델처럼 옵션에서 activation을 조절해주는 것이 아닌, 하나의 레이어로서 추가해주어야 한다. 

     base 모델보다 loss는 더 줄었고, accuracy는 더 높아진 것을 확인할 수 있었다. 이 결과를 바탕으로 활성함수는 relu와 leakyrelu로 고정시키고 나머지들을 바꿔보기로 했다.

     다음으로 진행한 tuning은 optimizer였다. adam, Nadam, RMSProp, Adagrad 등을 relu와 leakyrelu를 바탕으로 바꿔봤다.

     relu일 때는 adagrad를 제외하고는 거의 다 비슷비슷했다. 그래서 위의 adam모델을 사용하기로 임의로 결정했다. adagrad일 때는 accuracy가 0.973으로 다른 모델들에 비해 성능이 떨어지는 것을 확인할 수 있었다. 

     leakyrelu일 경우, optimizer를 Nadam으로 설정했을 때, accuracy가 가장 높아지는 것을 확인할 수 있었다. 0.989로 미약하게 나마 올랐다. relu일 때와 마찬가지로 adagrad일 때는 성능이 많이 떨어졌다.



In [ ]:
model6 = keras.Sequential([
                          keras.layers.Dense(512, activation = 'relu'),
                          keras.layers.Dropout(0.3),
                          keras.layers.Dense(256, activation = 'relu'),
                          keras.layers.Dropout(0.3),
                          keras.layers.Dense(128, activation = 'relu'),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(10, activation = 'softmax')
])

model6.compile(optimizer='Adagrad', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model6.fit(X_train, y_train, epochs = 20, batch_size = 100)
model6.evaluate(X_val, y_val)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 2s 58us/sample - loss: 0.5822 - accuracy: 0.8354
Epoch 2/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.2687 - accuracy: 0.9200
Epoch 3/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.2245 - accuracy: 0.9331
Epoch 4/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.2039 - accuracy: 0.9375
Epoch 5/20
33600/33600 [==============================] - 1s 42us/sample - loss: 0.1868 - accuracy: 0.9452
Epoch 6/20
33600/33600 [==============================] - 1s 42us/sample - loss: 0.1774 - accuracy: 0.9473
Epoch 7/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.1685 - accuracy: 0.9503
Epoch 8/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.1605 - accuracy: 0.9525
Epoch 9/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.1563 - accuracy: 0.9541
Epoch 10/20
33

[0.08411509088817097, 0.9734524]

In [ ]:
model7 = keras.Sequential([
                          keras.layers.Dense(512),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(256),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(128),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(10,activation = 'softmax')
])

model7.compile(optimizer='Nadam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model7.fit(X_train, y_train, epochs = 20, batch_size = 100)
model7.evaluate(X_val, y_val)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 3s 86us/sample - loss: 0.2336 - accuracy: 0.9323
Epoch 2/20
33600/33600 [==============================] - 2s 52us/sample - loss: 0.0807 - accuracy: 0.9757
Epoch 3/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0539 - accuracy: 0.9833
Epoch 4/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0424 - accuracy: 0.9863
Epoch 5/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0327 - accuracy: 0.9891
Epoch 6/20
33600/33600 [==============================] - 2s 52us/sample - loss: 0.0226 - accuracy: 0.9926
Epoch 7/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0235 - accuracy: 0.9927
Epoch 8/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0214 - accuracy: 0.9931
Epoch 9/20
33600/33600 [==============================] - 2s 50us/sample - loss: 0.0201 - accuracy: 0.9935
Epoch 10/20
33

[0.05386789367778766, 0.9895238]

In [ ]:
model9 = keras.Sequential([
                          keras.layers.Dense(512),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(256),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(128),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(10,activation = 'softmax')
])

model9.compile(optimizer='Adagrad', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model9.fit(X_train, y_train, epochs = 20, batch_size = 100)
model9.evaluate(X_val, y_val)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 2s 58us/sample - loss: 0.4593 - accuracy: 0.8771
Epoch 2/20
33600/33600 [==============================] - 1s 43us/sample - loss: 0.2200 - accuracy: 0.9353
Epoch 3/20
33600/33600 [==============================] - 1s 42us/sample - loss: 0.1871 - accuracy: 0.9456
Epoch 4/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.1683 - accuracy: 0.9511
Epoch 5/20
33600/33600 [==============================] - 1s 42us/sample - loss: 0.1568 - accuracy: 0.9544
Epoch 6/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.1477 - accuracy: 0.9568
Epoch 7/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.1409 - accuracy: 0.9601
Epoch 8/20
33600/33600 [==============================] - 1s 42us/sample - loss: 0.1394 - accuracy: 0.9598
Epoch 9/20
33600/33600 [==============================] - 1s 41us/sample - loss: 0.1330 - accuracy: 0.9613
Epoch 10/20
33

[0.08146662571245716, 0.97559524]

     위 결과를 바탕으로 relu는 adam을 고정시키고, leakyrelu는 Nadam을 고정시킨 후, weight initializer를 바꿔보기로 했다. 이번엔 여러 가지를 시도하진 않고 he_uniform과 he_normal만을 시도하기로 했다. relu와 leakyrelu의 경우, He를 사용했을 때 잘 된다는 점을 적용했다.

     relu 모형의 경우, he_normal과 he_uniform 모두 0.99의 accuracy를 보였으나 he_normal의 경우, he_uniform보다 더 loss가 작았다. leakyrelu의 경우, he_normal은 loss가 작았으나, accuracy의 경우, he_uniform이 더 작았다. 


In [ ]:
model11 = keras.Sequential([
                          keras.layers.Dense(512, activation = 'relu', kernel_initializer = 'he_normal'),
                          keras.layers.Dropout(0.3),
                          keras.layers.Dense(256, activation = 'relu', kernel_initializer = 'he_normal'),
                          keras.layers.Dropout(0.3),
                          keras.layers.Dense(128, activation = 'relu', kernel_initializer = 'he_normal'),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(10, activation = 'softmax', kernel_initializer = 'he_normal')
])

model11.compile(optimizer='adam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model11.fit(X_train, y_train, epochs = 20, batch_size = 100)
model11.evaluate(X_val, y_val)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 2s 54us/sample - loss: 0.2887 - accuracy: 0.9131
Epoch 2/20
33600/33600 [==============================] - 1s 39us/sample - loss: 0.1002 - accuracy: 0.9707
Epoch 3/20
33600/33600 [==============================] - 1s 39us/sample - loss: 0.0660 - accuracy: 0.9800
Epoch 4/20
33600/33600 [==============================] - 1s 39us/sample - loss: 0.0503 - accuracy: 0.9843
Epoch 5/20
33600/33600 [==============================] - 1s 39us/sample - loss: 0.0385 - accuracy: 0.9873
Epoch 6/20
33600/33600 [==============================] - 1s 39us/sample - loss: 0.0315 - accuracy: 0.9903
Epoch 7/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.0257 - accuracy: 0.9915
Epoch 8/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.0223 - accuracy: 0.9927
Epoch 9/20
33600/33600 [==============================] - 1s 40us/sample - loss: 0.0252 - accuracy: 0.9920
Epoch 10/20
33

[0.04616948565883961, 0.99035716]

In [ ]:
model13 = keras.Sequential([
                          keras.layers.Dense(512, kernel_initializer = 'he_normal'),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(256, kernel_initializer = 'he_normal'),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(128, kernel_initializer = 'he_normal'),
                          keras.layers.LeakyReLU(0.1,),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(10,activation = 'softmax', kernel_initializer = 'he_normal')
])

model13.compile(optimizer='Nadam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model13.fit(X_train, y_train, epochs = 20, batch_size = 100)
model13.evaluate(X_val, y_val)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 3s 78us/sample - loss: 0.2414 - accuracy: 0.9286
Epoch 2/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0864 - accuracy: 0.9734
Epoch 3/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0528 - accuracy: 0.9837
Epoch 4/20
33600/33600 [==============================] - 2s 52us/sample - loss: 0.0418 - accuracy: 0.9872
Epoch 5/20
33600/33600 [==============================] - 2s 52us/sample - loss: 0.0335 - accuracy: 0.9891
Epoch 6/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0258 - accuracy: 0.9920
Epoch 7/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0227 - accuracy: 0.9929
Epoch 8/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0202 - accuracy: 0.9936
Epoch 9/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0157 - accuracy: 0.9945
Epoch 10/20
33

[0.06407082769809481, 0.98702383]

     지금까지는 모두 Dropout을 사용했으나, 반대로 BatchNormalization을 사용해보기로 했다. 대체로 둘 중 하나만 사용하면 어느 정도 overfitting을 막을 수 있기 때문이다.

     위에서 설정한 두 모형의 dropout을 제거하고 BatchNormalization을 한 결과는 아래와 같다. dropout을 한 결과보다 accuracy가 살짝 떨어진 것을 볼 수 있었다.

In [ ]:
model14 = keras.Sequential([
                          keras.layers.Dense(512, activation = 'relu', kernel_initializer = 'he_uniform'),
                          keras.layers.BatchNormalization(),
                          keras.layers.Dense(256, activation = 'relu', kernel_initializer = 'he_uniform'),
                          keras.layers.BatchNormalization(),
                          keras.layers.Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'),
                          keras.layers.BatchNormalization(),
                          keras.layers.Dense(10, activation = 'softmax', kernel_initializer = 'he_uniform')
])

model14.compile(optimizer='adam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model14.fit(X_train, y_train, epochs = 20, batch_size = 100)
model14.evaluate(X_val, y_val)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 3s 81us/sample - loss: 0.1625 - accuracy: 0.9518
Epoch 2/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0434 - accuracy: 0.9876
Epoch 3/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0247 - accuracy: 0.9926
Epoch 4/20
33600/33600 [==============================] - 2s 50us/sample - loss: 0.0179 - accuracy: 0.9948
Epoch 5/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0158 - accuracy: 0.9948
Epoch 6/20
33600/33600 [==============================] - 2s 50us/sample - loss: 0.0127 - accuracy: 0.9959
Epoch 7/20
33600/33600 [==============================] - 2s 50us/sample - loss: 0.0127 - accuracy: 0.9962
Epoch 8/20
33600/33600 [==============================] - 2s 52us/sample - loss: 0.0110 - accuracy: 0.9965
Epoch 9/20
33600/33600 [==============================] - 2s 51us/sample - loss: 0.0114 - accuracy: 0.9964
Epoch 10/20
33

[0.05406718514980682, 0.9866667]

In [ ]:
model15 = keras.Sequential([
                          keras.layers.Dense(512, kernel_initializer = 'he_uniform'),
                          keras.layers.BatchNormalization(),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dense(256, kernel_initializer = 'he_uniform'),
                          keras.layers.BatchNormalization(),
                          keras.layers.LeakyReLU(0.1),
                          keras.layers.Dense(128, kernel_initializer = 'he_uniform'),
                          keras.layers.BatchNormalization(),
                          keras.layers.LeakyReLU(0.1,),
                          keras.layers.BatchNormalization(),
                          keras.layers.Dense(10,activation = 'softmax', kernel_initializer = 'he_uniform')
])

model15.compile(optimizer='Nadam', 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model15.fit(X_train, y_train, epochs = 20, batch_size = 100)
model15.evaluate(X_val, y_val)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 5s 146us/sample - loss: 0.1597 - accuracy: 0.9525
Epoch 2/20
33600/33600 [==============================] - 3s 75us/sample - loss: 0.0453 - accuracy: 0.9871
Epoch 3/20
33600/33600 [==============================] - 3s 76us/sample - loss: 0.0263 - accuracy: 0.9922
Epoch 4/20
33600/33600 [==============================] - 3s 75us/sample - loss: 0.0187 - accuracy: 0.9942
Epoch 5/20
33600/33600 [==============================] - 3s 76us/sample - loss: 0.0163 - accuracy: 0.9955
Epoch 6/20
33600/33600 [==============================] - 3s 75us/sample - loss: 0.0176 - accuracy: 0.9941
Epoch 7/20
33600/33600 [==============================] - 3s 77us/sample - loss: 0.0152 - accuracy: 0.9949
Epoch 8/20
33600/33600 [==============================] - 3s 77us/sample - loss: 0.0124 - accuracy: 0.9963
Epoch 9/20
33600/33600 [==============================] - 3s 77us/sample - loss: 0.0091 - accuracy: 0.9968
Epoch 10/20
3

[0.04897512786238145, 0.9892857]

     이 외에도 하이퍼 파라미터들을 튜닝하며 여러 모델들을 만들었다. 이 중 가장 accuracy가 높고, loss가 가장 낮은 model11을 최종적으로 선택했다. 이 모델에 X_test 데이터를 입력해 예측한 결과, 0.98814라는 accuracy를 얻을 수 있었다.

In [ ]:
predict = model11.predict(X_test)

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test.iloc[:,0]
sub['Category'] = np.argmax(predict,axis = 1)
sub.to_csv('submission.csv', index = False)

## CNN models

     여기까지가 예시로 준 모델을 따라서 진행한 NN모델이었다.

     필자는 kannada 데이터에 대해서 구글링을 하던 중, cnn에 대해서 알게 됐다. 이미지 데이터를 좀 더 잘 분류할 수 있는 NN 모델로 이를 사용하면 좀 더 높은 accuracy를 얻을 수 있다.

     아래 블로그에 있는 코드를 바탕으로 구현해보기로 했다.

     https://medium.com/@rahul.uday07/kannada-recognition-using-machine-learning-9e913aed0806
     https://kauri.io/multiclass-classification-example-with-convolution/145b4c03a55146cba2051f04e7f499e9/a

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

     위에서 진행한 데이터 전처리와는 조금 다른 모습이다. 위의 모델에서는 array 형식으로 받아드려서 분석을 해야하나, 이 모델을 사용하면 28*28의 이미지 형태로 데이터를 받아드려야 하기 때문이다.

In [ ]:
X_train1 = X_train.reshape(-1,28,28,1)
X_val1 = X_val.reshape(-1,28,28,1)
y_train1 = to_categorical(y_train)
y_val1 = to_categorical(y_val)

In [ ]:
model18 = models.Sequential()
model18.add(layers.Conv2D(32,(5,5), activation = 'relu', input_shape=(28,28,1)))
model18.add(layers.Conv2D(32,(5,5), activation = 'relu'))
model18.add(layers.MaxPooling2D((2,2)))
model18.add(layers.Dropout(0.25))

model18.add(layers.Conv2D(64, (5,5), activation = 'relu'))
model18.add(layers.Conv2D(64, (5,5), activation = 'relu'))
model18.add(layers.MaxPooling2D((2,2)))
model18.add(layers.Dropout(0.25))

model18.add(layers.Flatten())
model18.add(layers.Dense(256, activation = 'relu'))
model18.add(layers.Dropout(0.25))
model18.add(layers.Dense(10, activation = 'softmax'))

model18.compile(optimizer='adam', 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model18.fit(X_train1, y_train1, batch_size = 100, epochs = 20)
model18.evaluate(X_val1,y_val1)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 3s 80us/sample - loss: 0.3369 - accuracy: 0.8880
Epoch 2/20
33600/33600 [==============================] - 2s 57us/sample - loss: 0.0746 - accuracy: 0.9802
Epoch 3/20
33600/33600 [==============================] - 2s 58us/sample - loss: 0.0509 - accuracy: 0.9861
Epoch 4/20
33600/33600 [==============================] - 2s 57us/sample - loss: 0.0420 - accuracy: 0.9879
Epoch 5/20
33600/33600 [==============================] - 2s 58us/sample - loss: 0.0354 - accuracy: 0.9899
Epoch 6/20
33600/33600 [==============================] - 2s 57us/sample - loss: 0.0307 - accuracy: 0.9912
Epoch 7/20
33600/33600 [==============================] - 2s 57us/sample - loss: 0.0265 - accuracy: 0.9918
Epoch 8/20
33600/33600 [==============================] - 2s 57us/sample - loss: 0.0246 - accuracy: 0.9929
Epoch 9/20
33600/33600 [==============================] - 2s 58us/sample - loss: 0.0229 - accuracy: 0.9935
Epoch 10/20
33

[0.020044033112774612, 0.9954762]

In [ ]:
X_test1 = X_test.reshape(-1,28,28,1)

In [ ]:
predict18 = model18.predict(X_test1)

sub = pd.DataFrame()
sub['Id'] = test.iloc[:,0]
sub['Category'] = np.argmax(predict18,axis = 1)
sub.to_csv('submission1.csv', index = False)

     우선은 블로그에 있던 코드를 그대로 긁어서 모델을 만들어봤다. 확실히 이전에 위에서 진행한 basic 모델보다 accuracy가 확연하게 높아진 것을 볼 수 있었다. 

     그래서 이를 바탕으로 파라미터들을 조절해봤다. 활성 함수, optimizer 등을 바꾸거나 layer 층을 쌓거나, batchnormalization을 진행해봤다. 모델 평가까지만 해도 굉장히 높은 accuracy를 도출할 수 있었다. 하지만 실제 kaggle에 test 데이터로 평가한 값을 제출하면 accuracy가 0.1 안팎으로 나왔다.

     이는 train data에 한해서 너무 overfitting 된 것이 아닌가라고 조심스럽게 추측했다. 따라서 이를 기반으로 dropout을 조정하거나 option의 padding만 건드려보기로 했다.

In [ ]:
model19 = models.Sequential()
model19.add(layers.Conv2D(32,(5,5), padding = 'same', activation = 'relu', input_shape=(28,28,1)))
model19.add(layers.Conv2D(32,(5,5), padding = 'same', activation = 'relu'))
model19.add(layers.MaxPooling2D((2,2)))
model19.add(layers.Dropout(0.25))

model19.add(layers.Conv2D(64, (5,5), padding = 'same',activation = 'relu'))
model19.add(layers.Conv2D(64, (5,5), padding = 'same', activation = 'relu'))
model19.add(layers.MaxPooling2D((2,2)))
model19.add(layers.Dropout(0.25))

model19.add(layers.Flatten())
model19.add(layers.Dense(256, activation = 'relu'))
model19.add(layers.Dropout(0.5))
model19.add(layers.Dense(10, activation = 'softmax'))

model19.compile(optimizer='adam', 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model19.fit(X_train1, y_train1, batch_size = 100, epochs = 20)
model19.evaluate(X_val1,y_val1)

Train on 33600 samples
Epoch 1/20
33600/33600 [==============================] - 3s 98us/sample - loss: 0.2048 - accuracy: 0.9334
Epoch 2/20
33600/33600 [==============================] - 3s 83us/sample - loss: 0.0477 - accuracy: 0.9857
Epoch 3/20
33600/33600 [==============================] - 3s 82us/sample - loss: 0.0346 - accuracy: 0.9893
Epoch 4/20
33600/33600 [==============================] - 3s 83us/sample - loss: 0.0243 - accuracy: 0.9923
Epoch 5/20
33600/33600 [==============================] - 3s 82us/sample - loss: 0.0236 - accuracy: 0.9929
Epoch 6/20
33600/33600 [==============================] - 3s 83us/sample - loss: 0.0203 - accuracy: 0.9936
Epoch 7/20
33600/33600 [==============================] - 3s 83us/sample - loss: 0.0179 - accuracy: 0.9942
Epoch 8/20
33600/33600 [==============================] - 3s 83us/sample - loss: 0.0146 - accuracy: 0.9953
Epoch 9/20
33600/33600 [==============================] - 3s 83us/sample - loss: 0.0136 - accuracy: 0.9956
Epoch 10/20
33

[0.020880753475218294, 0.99595237]

In [ ]:
predict19 = model19.predict(X_test1)

sub = pd.DataFrame()
sub['Id'] = test.iloc[:,0]
sub['Category'] = np.argmax(predict19,axis = 1)
sub.to_csv('submission2.csv', index = False)

     옵션에 padding을 same으로 바꾸니 accuracy가 정말 미약하게 향상된 것을 알 수 있었다. 이 모델에서 더 나아가 Dropout 비율을 올리기로 했다. 

     dropout을 0.5까지 올리면서도 너무 높이 올리는 것이라고 의심하고 있었으나 결과는 더 좋아진 것을 보고 놀랐다. 학습 결과, loss는 더 떨어졌고, accuracy는 꽤 많이 상승했다. 

In [ ]:
model20 = models.Sequential()
model20.add(layers.Conv2D(32,(5,5), padding = 'same', activation = 'relu', input_shape=(28,28,1)))
model20.add(layers.Conv2D(32,(5,5), padding = 'same', activation = 'relu'))
model20.add(layers.MaxPooling2D((2,2)))
model20.add(layers.Dropout(0.5))

model20.add(layers.Conv2D(64, (5,5), padding = 'same',activation = 'relu'))
model20.add(layers.Conv2D(64, (5,5), padding = 'same', activation = 'relu'))
model20.add(layers.MaxPooling2D((2,2)))
model20.add(layers.Dropout(0.5))

model20.add(layers.Flatten())
model20.add(layers.Dense(256, activation = 'relu'))
model20.add(layers.Dropout(0.5))
model20.add(layers.Dense(10, activation = 'softmax'))

model20.compile(optimizer='adam', 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model20.fit(X_train1, y_train1, batch_size = 100, epochs = 20)
model20.evaluate(X_val1,y_val1)

Epoch 1/20
336/336 [==============================] - 7s 22ms/step - loss: 0.2511 - accuracy: 0.9186
Epoch 2/20
336/336 [==============================] - 7s 22ms/step - loss: 0.0611 - accuracy: 0.9823
Epoch 3/20
336/336 [==============================] - 7s 22ms/step - loss: 0.0451 - accuracy: 0.9859
Epoch 4/20
336/336 [==============================] - 7s 22ms/step - loss: 0.0375 - accuracy: 0.9893
Epoch 5/20
336/336 [==============================] - 7s 22ms/step - loss: 0.0326 - accuracy: 0.9892
Epoch 6/20
336/336 [==============================] - 7s 21ms/step - loss: 0.0300 - accuracy: 0.9905
Epoch 7/20
336/336 [==============================] - 7s 21ms/step - loss: 0.0241 - accuracy: 0.9926
Epoch 8/20
336/336 [==============================] - 7s 21ms/step - loss: 0.0249 - accuracy: 0.9924
Epoch 9/20
336/336 [==============================] - 7s 22ms/step - loss: 0.0233 - accuracy: 0.9929
Epoch 10/20
336/336 [==============================] - 7s 22ms/step - loss: 0.0189 - accura

[0.016192466020584106, 0.9964285492897034]

In [ ]:
predict20 = model20.predict(X_test1)

sub = pd.DataFrame()
sub['Id'] = test.iloc[:,0]
sub['Category'] = np.argmax(predict20,axis = 1)
sub.to_csv('submission3.csv', index = False)